In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130310  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,8.397054e+06
2018-02-28,6.845702e+06
2018-03-31,7.709179e+06
2018-04-30,8.448982e+06
2018-05-31,8.707443e+06
2018-06-30,7.703331e+06
2018-07-31,7.868991e+06
2018-08-31,1.023381e+07
2018-09-30,7.702752e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    8.397054e+06
 2018-02-28    6.845702e+06
 2018-03-31    7.709179e+06
 2018-04-30    8.448982e+06
 2018-05-31    8.707443e+06
 2018-06-30    7.703331e+06
 2018-07-31    7.868991e+06
 2018-08-31    1.023381e+07
 2018-09-30    7.702752e+06
 2018-10-31    7.984439e+06
 2018-11-30    7.222493e+06
 2018-12-31    7.168760e+06
 2019-01-31    8.619446e+06
 2019-02-28    7.371917e+06
 2019-03-31    7.796369e+06
 2019-04-30    7.991641e+06
 2019-05-31    9.027441e+06
 2019-06-30    7.936552e+06
 2019-07-31    8.591090e+06
 2019-08-31    1.119195e+07
 2019-09-30    7.748478e+06
 2019-10-31    8.786072e+06
 2019-11-30    8.303742e+06
 2019-12-31    8.148128e+06
 2020-01-31    8.758109e+06
 2020-02-29    6.282743e+06
 2020-03-31    5.892952e+06
 2020-04-30    5.666623e+06
 2020-05-31    7.429369e+06
 2020-06-30    7.817157e+06
 2020-07-31    8.762286e+06
 2020-08-31    1.123698e+07
 2020-09-30    8.035204e+06
 2020-10-31    9.641193e+06
 2020-11-30    9.801617e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.324011
p-value : 0.618117
Critical Values :
	1%: -3.653520
	5%: -2.957219
	10%: -2.617588


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.397065
p-value : 0.000300
Critical Values :
	1%: -3.646135
	5%: -2.954127
	10%: -2.615968


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1302.333, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1299.907, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1300.924, Time=0.03 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1297.718, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1299.277, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1301.818, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1298.614, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1300.485, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1302.289, Time=0.09 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1299.516, Time=0.08 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1301.274, Time=0.13 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1300.332, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1302.386, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1301.626, Time=0.13 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.29 sec
 ARIMA(3,0,0)(0

ARIMA(order=(3, 0, 2), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(3,0,2))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(3, 2)   Log Likelihood                -644.763
Method:                       css-mle   S.D. of innovations        1086204.470
Date:                Sat, 11 Sep 2021   AIC                           1303.526
Time:                        02:28:19   BIC                           1315.689
Sample:                             0   HQIC                          1307.984
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.335e+06   2.86e+05     29.157      0.000    7.77e+06     8.9e+06
ar.L1.y        0.7437      0.190      3.913      0.000       0.371       1.116
ar.L2.y       -0.8919      0.159     -5.620      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([9072342.22519225, 8006888.01860561]), array([1086204.46970885, 1130976.37717527]), array([[ 6943420.58471646, 11201263.86566802],
       [ 5790215.05197649, 10223560.98523472]]))
